In [1]:
import django_initializer

In [2]:
from ontrack.market.api.logic.endofdata import EndOfDayData
from ontrack.utils.datetime import DateTimeHelper as dt
from ontrack.utils.base.enum import ExchangeType, SettingKeyType
from ontrack.ta.utils._levels import get_eod_sr_levels, get_intraday_sr_levels
from ontrack.ta.core import sanitize
import ontrack.ta as ta
from operator import itemgetter
from ontrack.utils.numbers import NumberHelper as nh
from IPython.display import display, Markdown

import pandas as pd
import yfinance as yf
import matplotlib.dates as mpl_dates

def get_stock_price(symbol, period="2y", interval="1d", start_date=None, end_date=None):
  df = yf.download(tickers=symbol, interval=interval, period=period, start=start_date, end=end_date)
  df['Date'] = pd.to_datetime(df.index)
  df['Date'] = df['Date'].apply(mpl_dates.date2num)
  df = df.loc[:,['Date', 'Open', 'High', 'Low', 'Close']]
  return df

In [3]:
import numpy as np
from pandas import Timestamp
import math
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.pyplot as plt

In [4]:
#symbol = 'TCS.NS'
#symbol = 'IDFCFIRSTB.NS'
#symbol = '^NSEBANK'
#symbol = "^NSEI"
symbol = "MSFT"
df_y = get_stock_price(symbol, "2y", "1d")
df_m = get_stock_price(symbol, "2y", "1mo")
df_w = get_stock_price(symbol, "2y", "1wk")
df_d = get_stock_price(symbol, "1y", "1d")
df_h = get_stock_price(symbol, "6mo", "1h")

df_y = sanitize(df_y)
df_m = sanitize(df_m)
df_w = sanitize(df_w)
df_d = sanitize(df_d)
df_h = sanitize(df_h)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [5]:
obj = EndOfDayData(ExchangeType.NSE)
result= obj.populate_indicators(df_y)
df = result[0]                
last_record = df.ta.second_last_record
eod_levels = get_eod_sr_levels(df, [df_m, df_w, df_h])

DEBUG 2022-11-18 20:08:07,679 logger 7412 5660 [c:\Sachin\OST-API\ontrack\utils\config.py.__get_config(16)]: Reading configuration [C:\Sachin\OST-API\config\app/urlconfig.json].


[X] Ooops!!! It's True, the series 'quantity_per_trade' was not found in date, open, high, low, close, SMA_200, SMA_100, SMA_50, EMA_200, EMA_100, EMA_50, EMA_5, EMA_9, EMA_13, BBL_20_1.5, BBM_20_1.5, BBU_20_1.5, BBB_20_1.5, BBP_20_1.5, CPR_TC, CPR_PIVOT, CPR_BC, CPR_R3, CPR_R2, CPR_R1, CPR_S3, CPR_S2, CPR_S1, CPR, CDL_CONSOLIDATED
[X] Ooops!!! It's True, the series 'volume' was not found in date, open, high, low, close, SMA_200, SMA_100, SMA_50, EMA_200, EMA_100, EMA_50, EMA_5, EMA_9, EMA_13, BBL_20_1.5, BBM_20_1.5, BBU_20_1.5, BBB_20_1.5, BBP_20_1.5, CPR_TC, CPR_PIVOT, CPR_BC, CPR_R3, CPR_R2, CPR_R1, CPR_S3, CPR_S2, CPR_S1, CPR, CDL_CONSOLIDATED
[X] Ooops!!! It's True, the series 'delivery_percentage' was not found in date, open, high, low, close, SMA_200, SMA_100, SMA_50, EMA_200, EMA_100, EMA_50, EMA_5, EMA_9, EMA_13, BBL_20_1.5, BBM_20_1.5, BBU_20_1.5, BBB_20_1.5, BBP_20_1.5, CPR_TC, CPR_PIVOT, CPR_BC, CPR_R3, CPR_R2, CPR_R1, CPR_S3, CPR_S2, CPR_S1, CPR, CDL_CONSOLIDATED
[X] Ooops

In [6]:
def candle_stick(df):
    strategy = ta.Strategy(
        name="intraday",
        ta=[
            {
                "kind": "cdl_pattern",
                "name": "all",
                "consolidated": True,
                "append": True,
            }
        ],
    )
    df.ta.cores = 0
    df.ta.sanitize()
    df.ta.strategy(strategy)

    row = df.ta.second_last_record

    cdl_rows = []
    cdl_string = row["CDL_CONSOLIDATED"].strip()
    value_ = "| "
    if cdl_string:
        for cdl in cdl_string.split(";"):
            cdl_rs = cdl.split("|")
            name = cdl_rs[0]
            score = nh.str_to_float(cdl_rs[1])
            sentiment = "BULLISH" if score > 0 else "BEARISH"
            sentiment_html = f"<b style='color:#FF0000'>BEARISH</b> {name} {score}" if score < 0 else f"<b style='color:#008000'>BULLISH</b>  {name} {score}"

            cdl_row = {}
            cdl_row["rank"] = 0
            cdl_row["name"] = name
            cdl_row["sentiment"] = sentiment
            cdl_row["score"] = score
            cdl_rows.append(cdl_row)
            
            value_ = f"{value_}{sentiment_html}| "

    cdl_rows = sorted(cdl_rows, key=itemgetter("rank"), reverse=False)
    return cdl_string, cdl_rows, value_

def print_result(levels, price):
    print("--------------------")
    records_below = [x for x in levels if x['is_below']]
    records_above = [x for x in levels if not x['is_below']]
    record = records_below[-3]
    print(f"Point: {record['point']}, Min: {record['min_point']}, Max: {record['max_point']}")
    print(f"Types: {set(record['types'])}")
    print("--------------------")
    record = records_below[-2]
    print(f"Point: {record['point']}, Min: {record['min_point']}, Max: {record['max_point']}")
    print(f"Types: {set(record['types'])}")
    print("--------------------")
    record = records_below[-1]
    print(f"Point: {record['point']}, Min: {record['min_point']}, Max: {record['max_point']}")
    print(f"Types: {set(record['types'])}")
    print("--------------------")
    display(Markdown(f"**Current Price: {price}**"))
    print("--------------------")
    record = records_above[0]
    print(f"Point: {record['point']}, Min: {record['min_point']}, Max: {record['max_point']}")
    print(f"Types: {set(record['types'])}")
    print("--------------------")
    record = records_above[1]
    print(f"Point: {record['point']}, Min: {record['min_point']}, Max: {record['max_point']}")
    print(f"Types: {set(record['types'])}")
    print("--------------------")
    record = records_above[2]
    print(f"Point: {record['point']}, Min: {record['min_point']}, Max: {record['max_point']}")
    print(f"Types: {set(record['types'])}")
    print("--------------------")
    
def print_value(message):
    display(Markdown(f"**{message}**"))
    
def print_cpr_location(price, prev_day):
    if price >= prev_day["CPR_R3"]:
        print_value("ABOVE R3")
    elif price >= prev_day["CPR_R2"]:
        print_value("BETWEEN R2 AND R3")
    elif price >= prev_day["CPR_R1"]:
        print_value("BETWEEN R1 AND R2")
    elif price >= prev_day["CPR_TC"]:
        print_value("BETWEEN R1 AND TC")
    elif price >= prev_day["CPR_PIVOT"]:    
        print_value("BETWEEN PIVOT AND TC")
    elif prev_day["CPR_BC"] < price:
        print_value("BETWEEN PIVOT AND BC")
    elif prev_day["CPR_S1"] < price:
        print_value("BETWEEN BC AND S1")
    elif prev_day["CPR_S2"] < price:
        print_value("BETWEEN S2 AND S1")
    elif prev_day["CPR_S3"] < price:
        print_value("BETWEEN S3 AND S2")
    elif prev_day["CPR_S3"] > price:
        print_value("BELOW S3")
    


In [8]:
df_d = get_stock_price(symbol, "3d", "1d")
df_h = get_stock_price(symbol, "3d", "1h")
df_15m = get_stock_price(symbol, "2d", "15m")
df_5m = get_stock_price(symbol, "2d", "5m")
df_1m = get_stock_price(symbol, "1d", "1m")

df_d = sanitize(df_d)
df_h = sanitize(df_h)
df_15m = sanitize(df_15m)
df_5m = sanitize(df_5m)
df_1m = sanitize(df_1m)

price = df_15m.iloc[-1]["close"]
levels = get_intraday_sr_levels(eod_levels, df_d, [df_5m, df_15m], price)

print_cpr_location(price, last_record)
print_result(levels, price)

r = candle_stick(df_h)
display(Markdown(f'**1 hour:** {r[2]}'))

r = candle_stick(df_15m)
display(Markdown(f'**15 minutes:** {r[2]}'))

r = candle_stick(df_5m)
display(Markdown(f'**5 minutes:** {r[2]}'))

r = candle_stick(df_1m)
display(Markdown(f'**1 minute:** {r[2]}'))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


**BETWEEN PIVOT AND BC**

--------------------
Point: 239.17, Min: 238.6, Max: 239.17
Types: {'SR_FCP', 'CPR_S2', 'SR_WSM'}
--------------------
Point: 239.71, Min: 239.21, Max: 239.71
Types: {'SR_FCP', 'CPR_S1', 'SR_WSM'}
--------------------
Point: 240.42, Min: 239.92, Max: 240.42
Types: {'C_D_L', 'SR_FCP', 'SR_WSM'}
--------------------


**Current Price: 240.5399932861328**

--------------------
Point: 240.54, Min: 240.54, Max: 241.06
Types: {'C_D_L', 'SR_FCP', 'C_D_C', 'C_M_C', 'C_W_C', 'EMA_50', 'SR_WSM'}
--------------------
Point: 241.27, Min: 241.27, Max: 241.8
Types: {'CPR_PIVOT', 'CPR_TC', 'SR_FCP', 'SR_WSM'}
--------------------
Point: 241.99, Min: 241.99, Max: 242.33
Types: {'SR_FCP', 'P_D_C', 'C_W_O', 'CPR_BC', 'SR_WSM'}
--------------------


**1 hour:** | <b style='color:#FF0000'>BEARISH</b> CDL_DARKCLOUDCOVER -100.0| <b style='color:#FF0000'>BEARISH</b> CDL_HIKKAKE -100.0| <b style='color:#FF0000'>BEARISH</b> CDL_LONGLINE -100.0| 

**15 minutes:** | <b style='color:#FF0000'>BEARISH</b> CDL_ENGULFING -100.0| 

**5 minutes:** | 

**1 minute:** | 